# Status #

- [x] Outline
- [x] Introduction
- [ ] Exercise 1
  - [x] Code
  - [x] Discussion
  - [ ] Checking
- [ ] Exercise 2
  - [x] Code
  - [x] Discussion
  - [ ] Checking
- [x] Exercise 3
  - [x] Code
  - [x] Discussion
  - [x] Checking
- [ ] Exercise 4
  - [ ] Code
  - [x] Discussion
  - [ ] Checking
- [x] Conclusion

# Introduction #

In these exercises, you'll conclude the feature extraction begun in Exercise 2, explore how invariance is created by maximum pooling, and then look at a different kind of pooling: *average* pooling.

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex3 import *

from cv_prelude import *

### 1) Apply Pooling to Condense

Run this cell to get back to where you left off in the previous lesson. There is a kernel predefined for you, but feel free to change it to the one you were using before or something else, if you like.

In [ ]:
# Read image
image_path = '/kaggle/input/computer-vision-resources/car_feature.jpg'
image = tf.io.read_file(image_path)
image = tf.io.decode_jpeg(image)

# Embossing kernel
kernel = tf.constant([
    [-2, -1, 0],
    [-1, 1, 1],
    [0, 1, 2],
])

# Reformat for batch compatibility.
image = tf.image.convert_image_dtype(image, dtype=tf.float32)
image = tf.expand_dims(image, axis=0)
kernel = tf.reshape(kernel, [*kernel.shape, 1, 1])
kernel = tf.cast(kernel, dtype=tf.float32)

image_filter = tf.nn.conv2d(
    input=image,
    filters=kernel,
    strides=1,
    padding='SAME',
)

image_detect = tf.nn.relu(image_filter)

# Show what we have so far
plt.figure(figsize=(12, 6))
plt.subplot(131)
plt.imshow(tf.squeeze(image), cmap='gray')
plt.axis('off')
plt.title('Input')
plt.subplot(132)
plt.imshow(tf.squeeze(image_filter))
plt.axis('off')
plt.title('Filter')
plt.subplot(133)
plt.imshow(tf.squeeze(image_detect))
plt.axis('off')
plt.title('Detect')
plt.show();

Now for the last step in the sequence. Apply maximum pooling with `tf.nn.pool`. Use the same parameter values that we used in the example from the tutorial.

In [ ]:
# YOUR CODE HERE
image_condense = ____

q_1.check()

In [ ]:
#%%RM_IF(PROD)%%
image_condense = tf.nn.pool(
    input=image_detect, # image in the Detect step above
    window_shape=(2, 2),
    pooling_type='MAX',
    # we'll see what these do in the next lesson!
    strides=(2, 2),
    padding='SAME',
)
q_1.assert_check_passed()

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_1.hint()
#_COMMENT_IF(PROD)_
q_1.solution()

Run the next cell to see what maximum pooling did to the feature!

In [ ]:
plt.imshow(
    # Reformat for plotting
    tf.squeeze(image_detect)
)
plt.axis('off')
plt.show();

### 2) Explore Invariance

This next code cell will randomly apply a small shift to the circle and then condense the image several times with maximum pooling. Run this cell a few times and make note of the feature produced at the end.

In [ ]:
REPEATS = 4
SIZE = [64, 64]

# Create a randomly shifted circle
image = visiontools.circle(SIZE, r_shrink=4, val=1)
image = tf.expand_dims(image, axis=-1)
image = visiontools.random_transform(image, jitter=3, fill_method='replicate')
image = tf.squeeze(image)

plt.figure(figsize=(16, 4))
plt.subplot(1, REPEATS+1, 1)
plt.imshow(image, vmin=0, vmax=1)
plt.title("Original\nShape: {}x{}".format(image.shape[0], image.shape[1]))
plt.axis('off')

# Now condense with maximum pooling several times
for i in range(REPEATS):
    ax = plt.subplot(1, REPEATS+1, i+2)
    image = tf.reshape(image, [1, *image.shape, 1])
    image = tf.nn.pool(image, window_shape=(2,2), strides=(2, 2), padding='SAME', pooling_type='MAX')
    image = tf.squeeze(image)
    plt.imshow(image, vmin=0, vmax=1)
    plt.title("MaxPool {}\nShape: {}x{}".format(i+1, image.shape[0], image.shape[1]))
    plt.axis('off')

If you were wanting to teach a convolutional classifier to recognize circles, would it be helpful to include all of these "shifted" circles in your dataset?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_2.hint()
#_COMMENT_IF(PROD)_
q_2.solution()

In Lesson 6, we'll explore this idea of invariance more when we learn about **Data Augmentation**.

### 3) What about Average Pooling?

In this exercise, we'll look at an alternative to maximum pooling that was once very popular: **average pooling**. An `AvgPool2D` layer operates the same way as `MaxPool2D`, but replaces a patch of pixels by their *average* instead of their maximum value.

Run the following cell to see the effect of average pooling to a feature over several iterations.

In [ ]:
REPEATS = 4

SIZE = [32, 32]
image = visiontools.circle(SIZE)
image = tf.reshape(image, [1, *image.shape, 1])

plt.figure(figsize=(16, 4))
plt.subplot(1, REPEATS+1, 1)
plt.imshow(tf.squeeze(image), vmin=0, vmax=1)
plt.title("Original\nShape: {}x{}".format(image.shape[0], image.shape[1]))
plt.axis('off')
for i in range(REPEATS):
    ax = plt.subplot(1, REPEATS+1, i+2)
    image = tf.nn.avg_pool(image, ksize=2, strides=2, padding='SAME')
    plt.imshow(tf.squeeze(image), vmin=0, vmax=1)
    plt.title("MaxPool {}\nShape: {}x{}".format(i+1, image.shape[1], image.shape[2]))
    plt.axis('off')

What happened? All else equal, what do you think would happen if you replaced maximum pooling with average pooling in a convolutional classifier? Would you expect its performance to improve?

In [ ]:
# Lines below will give you a hint or solution code
#_COMMENT_IF(PROD)_
q_3.hint()
#_COMMENT_IF(PROD)_
q_3.solution()

### 4) What about Global Average Pooling?

We mentioned in the previous exercise that average pooling has largely been superceeded by maximum pooling within the convolutional base. There is, however, a kind of average pooling that is still widely used in the *head* of a convnet. This is **global average pooling**. A `GlobalAvgPool2D` layer is often used as an alternative to some or all of the hidden `Dense` layers in the head of the network, like so:

```
model = keras.Sequential([
    pretrained_base,
    layers.GlobalAvgPool2D(),
    layers.Dense(1, activation='sigmoid'),
])
```

What is this layer doing? Notice that we no longer have the `Flatten` layer that usually comes after the base to transform the 2D feature data to 1D data needed by the classifier. Now the `GlobalAvgPool2D` layer is serving this function. But, instead of "unstacking" the feature (like `Flatten`), it simply replaces the entire feature map with its average value. Though very destructive, it often works quite well and has the advantage of reducing the number of parameters in the model.

This next cell will show you a typical set of feature maps produced by the base of VGG16, and what happens when gloval average pooling is applied to them. Run the following cell to see `GlobalAvgPool2D` in action.

In [ ]:


q_4.a.check()

Since each of the $X \times X$ feature maps was reduced to a single value, global pooling reduced the number of parameters needed to represent these features by a factor of X -- a substantial savings!

Global average pooling is used in a number of modern convnets -- <!-- TODO -->, and is a standard option with the predefined models in `tf.keras.applications`. If you're creating a convolutional classifier, it's worth trying out!

# Conclusion #

In this lesson we explored the final operation in the feature extraction process: **condensing** with **maximum pooling**. Pooling is one of the essential features of convolutional networks and helps provide them with some of their characteristic advantages: efficiency with visual data, reduced parameter size compared to dense networks, translation invariance. We've seen that it's used not only in the base during feature extraction, but also can be used in the head during classification. Understanding it is essential to a full understanding of convnets.

In the next lesson, we'll conclude our discussion of the feature extraction operations with **moving windows**, the typical way of describing how the convolution and pooling operations scan over an image. We'll describe here the final two parameters in the `Conv2D` and `MaxPool2D` layers: `strides` and `padding`.